<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.4: Sentiment Analysis
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

Based on the video tutorial **Text Classification with Machine Learning,SpaCy and Scikit(Sentiment Analysis)** by **Jesse E. Agbe (JCharis)**.

## Data Source: UCI
### UCI - Machine Learning Repository
- Center for Machine Learning and Intelligent Systems

The [**UCI Machine Learning Repository**](http://archive.ics.uci.edu/ml/about.html) is a collection of databases, domain theories, and data generators that are used by the machine learning community for the empirical analysis of machine learning algorithms.

### Dataset
- [Sentiment Labelled Sentences Data Set](http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences)

### Abstract
The dataset contains sentences labelled with positive or negative sentiment.

- Data Set Characteristics: Text
- Number of Instances: 3000
- Area: N/A
- Attribute Characteristics: N/A
- Number of Attributes: N/A
- Date Donated: 2015-05-30
- Associated Tasks: Classification
- Missing Values? N/A
- Number of Web Hits: 102584

### Source
Dimitrios Kotzias dkotzias '@' ics.uci.edu

### Data Set Information
This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015

Please cite the paper if you want to use it :)

It contains sentences labelled with positive or negative sentiment.

### Format
sentence &lt;tab&gt; score &lt;newline&gt;

### Details
Score is either 1 (for positive) or 0 (for negative)

The sentences come from three different websites/fields:
- imdb.com
- amazon.com
- yelp.com

For each website, there exist **500 positive** and **500 negative** sentences. Those were selected randomly for larger datasets of reviews.

We attempted to select sentences that have a clearly positive or negative connotaton, the goal was for no neutral sentences to be selected.

For the full datasets look:

- **imdb**: Maas et. al., 2011 _Learning word vectors for sentiment analysis_
- **amazon**: McAuley et. al., 2013 _Hidden factors and hidden topics: Understanding rating dimensions with review text_
- **yelp**: [Yelp dataset challenge](http://www.yelp.com/dataset_challenge)


### Attribute Information
The attributes are text sentences, extracted from reviews of products, movies, and restaurants

### Relevant Papers
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

### Citation Request
**From Group to Individual Labels using Deep Features**, Kotzias et. al,. KDD 2015

## Import libraries

In [1]:
## Import Libraries
import pandas as pd

import regex as re
import spacy

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import warnings
warnings.filterwarnings('ignore')

## Load data

Load Yelp, Amazon and Imdb Data.

Hint: Source is separated by <tab>s and has no headers.

In [4]:
yelp_text = 'yelp_labelled.txt'
imdb_text = 'imdb_labelled_fixed.txt'
amazon_text = 'amazon_cells_labelled.txt'

# ANSWER

df_yelp = pd.read_csv('/Users/ayano/Desktop/Data Science & AI/csv/yelp_labelled.txt', header = None, sep = '\t')
df_imdb = pd.read_csv('/Users/ayano/Desktop/Data Science & AI/csv/imdb_labelled_fixed.txt', header = None, sep = '\t')
df_amazon = pd.read_csv('/Users/ayano/Desktop/Data Science & AI/csv/amazon_cells_labelled.txt', header = None, sep = '\t')
dfs = {'yelp': df_yelp, 'imdb': df_imdb, 'amazon': df_amazon}

## Inspect the data

Check your datasets.

In [11]:
# ANSWER

# データを辞書にまとめる
dfs = {'yelp': df_yelp, 'imdb': df_imdb, 'amazon': df_amazon}

# Inspect the Data
for ds, df in dfs.items():
    print(f'Dataset: {ds} [{df.shape[0]} row(s) x {df.shape[1]} col(s)]')
    print(df.head())  # 最初の数行を表示
    print('\nSummary:')
    print(df.info())  # データの概要を表示
    print('-' * 75)

Dataset: yelp [1000 row(s) x 2 col(s)]
                                                   0  1
0                           Wow... Loved this place.  1
1                                 Crust is not good.  0
2          Not tasty and the texture was just nasty.  0
3  Stopped by during the late May bank holiday of...  1
4  The selection on the menu was great and so wer...  1

Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1000 non-null   object
 1   1       1000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 15.8+ KB
None
---------------------------------------------------------------------------
Dataset: imdb [1000 row(s) x 2 col(s)]
                                                   0  1
0  A very, very, very slow-moving, aimless movie ...  0
1  Not sure who was more lost - the flat characte...  0
2  Attempting artiness with

## Merge the data

Merge all three datasets.

In [13]:
# ANSWER

# 列名を割り当て
df_yelp.columns = ['text', 'sentiment']
df_imdb.columns = ['text', 'sentiment']
df_amazon.columns = ['text', 'sentiment']

# データを結合
merged_df = pd.concat([df_yelp, df_imdb, df_amazon], ignore_index=True)

# Inspect the Merged Data
print(f'Merged Dataset: [{merged_df.shape[0]} row(s) x {merged_df.shape[1]} col(s)]')
print(merged_df.head())  # 最初の数行を表示
print('\nSummary:')
print(merged_df.info())  # データの概要を表示
print('\nLabel Distribution:')
print(merged_df['sentiment'].value_counts())  # 感情ラベルの分布を表示
print('-' * 75)


Merged Dataset: [3000 row(s) x 2 col(s)]
                                                text  sentiment
0                           Wow... Loved this place.          1
1                                 Crust is not good.          0
2          Not tasty and the texture was just nasty.          0
3  Stopped by during the late May bank holiday of...          1
4  The selection on the menu was great and so wer...          1

Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3000 non-null   object
 1   sentiment  3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB
None

Label Distribution:
1    1500
0    1500
Name: sentiment, dtype: int64
---------------------------------------------------------------------------


## Export the data

Export merged datasets to as csv file.

In [14]:
# ANSWER

# CSVファイルにデータをエクスポート
merged_df.to_csv('/Users/ayano/Desktop/Data Science & AI/csv/merged_dataset.csv', index=False)

## Prepare the stage
- Load spaCy

In [16]:
nlp = spacy.load('en_core_web_sm')

## Prepare the text
All the text handling and preparation concerned with the changes and modifications from the raw source text to a format that will be used for the actual processing, things like:
- handle encoding
- handle extraneous and international characters
- handle symbols
- handle metadata and embedded information
- handle repetitions (such multiple spaces or newlines)

Clean text.

In [17]:
def clean_text(text):
    # reduce multiple spaces and newlines to only one
    text = re.sub(r'(\s\s+|\n\n+)', r'\1', text)
    # remove double quotes
    text = re.sub(r'"', '', text)

    return text

In [18]:
# ANSWER
df['text'] = df['text'].apply(clean_text)

## Work the text
Concern with the meaning and the substance of the content to extract actual information.

Hint: Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.

In [19]:
def convert_text(text):
    '''
    Use techniques learned in previous labs. Remove StopWords, Punctuation, Lemmatize etc.
    '''
    return text

In [20]:
%%time
df['short'] = df['text'].apply(convert_text)

CPU times: user 2.67 ms, sys: 6.62 ms, total: 9.29 ms
Wall time: 14 ms


In [21]:
df.sample(10)

,text,sentiment,short
494,Pretty piece of junk.,0,Pretty piece of junk.
141,"During several different 2 minute calls, I exp...",0,"During several different 2 minute calls, I exp..."
688,Too bad you have to pay up to $$$ a month for ...,0,Too bad you have to pay up to $$$ a month for ...
513,I got the aluminum case for my new Palm VX and...,1,I got the aluminum case for my new Palm VX and...
29,Doesn't hold charge.,0,Doesn't hold charge.
433,I highly recommend this modest priced cellular...,1,I highly recommend this modest priced cellular...
280,"Good product, good seller.",1,"Good product, good seller."
387,The best phone in market :).,1,The best phone in market :).
514,"very good product, well made.",1,"very good product, well made."
383,After trying many many handsfree gadgets this ...,1,After trying many many handsfree gadgets this ...


## Modelling

In [22]:
# helper function to show results and charts
def show_summary_report(actual, prediction, probabilities):

    if isinstance(actual, pd.Series):
        actual = actual.values.astype(int)
    prediction = prediction.astype(int)

    accuracy_ = accuracy_score(actual, prediction)
    precision_ = precision_score(actual, prediction)
    recall_ = recall_score(actual, prediction)
    roc_auc_ = roc_auc_score(actual, probabilities)

    print('Accuracy : %.4f [TP / N] Proportion of predicted labels that match the true labels. Best: 1, Worst: 0' % accuracy_)
    print('Precision: %.4f [TP / (TP + FP)] Not to label a negative sample as positive.        Best: 1, Worst: 0' % precision_)
    print('Recall   : %.4f [TP / (TP + FN)] Find all the positive samples.                     Best: 1, Worst: 0' % recall_)
    print('ROC AUC  : %.4f                                                                     Best: 1, Worst: < 0.5' % roc_auc_)
    print('-' * 107)
    print('TP: True Positives, FP: False Positives, TN: True Negatives, FN: False Negatives, N: Number of samples')

    # Confusion Matrix
    mat = confusion_matrix(actual, prediction)

    # Precision/Recall
    precision, recall, _ = precision_recall_curve(actual, probabilities)
    average_precision = average_precision_score(actual, probabilities)
    
    # Compute ROC curve and ROC area
    fpr, tpr, _ = roc_curve(actual, probabilities)
    roc_auc = auc(fpr, tpr)


    # plot
    fig, ax = plt.subplots(1, 3, figsize = (18, 6))
    fig.subplots_adjust(left = 0.02, right = 0.98, wspace = 0.2)

    # Confusion Matrix
    sns.heatmap(mat.T, square = True, annot = True, fmt = 'd', cbar = False, cmap = 'Blues', ax = ax[0])

    ax[0].set_title('Confusion Matrix')
    ax[0].set_xlabel('True label')
    ax[0].set_ylabel('Predicted label')
    
    # Precision/Recall
    step_kwargs = {'step': 'post'}
    ax[1].step(recall, precision, color = 'b', alpha = 0.2, where = 'post')
    ax[1].fill_between(recall, precision, alpha = 0.2, color = 'b', **step_kwargs)
    ax[1].set_ylim([0.0, 1.0])
    ax[1].set_xlim([0.0, 1.0])
    ax[1].set_xlabel('Recall')
    ax[1].set_ylabel('Precision')
    ax[1].set_title('2-class Precision-Recall curve')

    # ROC
    ax[2].plot(fpr, tpr, color = 'darkorange', lw = 2, label = 'ROC curve (AUC = %0.2f)' % roc_auc)
    ax[2].plot([0, 1], [0, 1], color = 'navy', lw = 2, linestyle = '--')
    ax[2].set_xlim([0.0, 1.0])
    ax[2].set_ylim([0.0, 1.0])
    ax[2].set_xlabel('False Positive Rate')
    ax[2].set_ylabel('True Positive Rate')
    ax[2].set_title('Receiver Operating Characteristic')
    ax[2].legend(loc = 'lower right')

    plt.show()
    
    return (accuracy_, precision_, recall_, roc_auc_)

In [23]:
# Features and Labels
X = df['short']
y = df['sentiment']

# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Use Bag-of-Words

In [24]:
# create a matrix of word counts from the text
counts = CountVectorizer()

In [25]:
# do the actual counting
A = counts.fit_transform(X_train, y_train)

In [26]:
# create a classifier using SVC
classifier = SVC(kernel='linear', probability=True)

In [27]:
# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

SVC(kernel='linear', probability=True)

In [32]:
# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
B = counts.transform(X_test)


In [33]:
# make predictions based on the test data
predictions = classifier.predict(B.toarray())
predictions

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1])

In [34]:
# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, predictions))

Accuracy: 0.8450


## Repeat using TF-IDF
TF-IDF = Term Frequency - Inverse Document Frequency

In [36]:
# create a matrix of word counts from the text
# use TF-IDF
tfidf = TfidfVectorizer()
# do the actual counting
A = tfidf.fit_transform(X_train, y_train)

# train the classifier with the training data
classifier.fit(A.toarray(), y_train)

# do the transformation for the test data
# NOTE: use `transform()` instead of `fit_transform()`
B = tfidf.transform(X_test)

# make predictions based on the test data
predictions = classifier.predict(B.toarray())

# check the accuracy
print('Accuracy: %.4f' % accuracy_score(y_test, predictions))

Accuracy: 0.8650


## Repeating it all for comparision
Repeating the whole lot in one big block

Find 'Accuracy', 'Precision', 'Recall', 'ROC_AUC' using CountVectorizer and TfidfVectorizer and keep the result in a dataframe.

In [38]:
# ANSWER

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# Assuming you have a DataFrame 'df' with 'text' and 'label' columns
# Replace 'df' with your actual DataFrame if needed

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)

# Function to train and evaluate the model
def train_and_evaluate(vectorizer, X_train, X_test, y_train, y_test):
    # Transform text data into features
    A_train = vectorizer.fit_transform(X_train)
    A_test = vectorizer.transform(X_test)

    # Create a classifier using SVC
    classifier = SVC(kernel='linear', probability=True)
    
    # Train the classifier with the training data
    classifier.fit(A_train.toarray(), y_train)
    
    # Make predictions based on the test data
    predictions = classifier.predict(A_test.toarray())

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, classifier.predict_proba(A_test.toarray())[:, 1])

    return {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'ROC_AUC': roc_auc}

# Create a DataFrame to store the results
results_df = pd.DataFrame(columns=['Vectorizer', 'Accuracy', 'Precision', 'Recall', 'ROC_AUC'])

# Train and evaluate with CountVectorizer
count_vectorizer_results = train_and_evaluate(CountVectorizer(), X_train, X_test, y_train, y_test)
results_df = results_df.append({'Vectorizer': 'CountVectorizer', **count_vectorizer_results}, ignore_index=True)

# Train and evaluate with TfidfVectorizer
tfidf_vectorizer_results = train_and_evaluate(TfidfVectorizer(), X_train, X_test, y_train, y_test)
results_df = results_df.append({'Vectorizer': 'TfidfVectorizer', **tfidf_vectorizer_results}, ignore_index=True)

# Display the results
print(results_df)


        Vectorizer  Accuracy  Precision    Recall   ROC_AUC
0  CountVectorizer     0.845   0.858491  0.850467  0.915285
1  TfidfVectorizer     0.865   0.908163  0.831776  0.937795




---



---



> > > > > > > > > © 2023 Institute of Data


---



---



